ETL Lab Exercise: NYC Yellow Taxi Trip Dataset (Public CSV Download)
Problem Statement
In this exercise, you'll build an ETL (Extract-Transform-Load) pipeline in Google Colab, using real-world trip data from New York City's iconic yellow taxis.


Tasks

1. Extract

Download:
Download a recent sample NYC Yellow Taxi Trip data CSV from this public direct URL:
https://data.cityofnewyork.us/resource/kxp8-n2sj.csv?$limit=5000
Load:
Load the CSV file into a pandas DataFrame.

2. Transform

Clean:
Handle missing or inconsistent values in key columns (like fare amount, trip distance, passenger count).
Convert date columns (e.g., pickup/dropoff datetime) to proper datetime format.
Remove records that aren't plausible (e.g., zero or negative trip distance or fare).
Feature Engineering:
Compute trip duration in minutes.
Create a new feature called Tip_Percent as tip_amount/total_amount * 100.
Categorize each ride by time of day ("Morning", "Afternoon", "Evening", "Night") using the pickup timestamp.

3. Load

SQLite Storage:
Store the transformed DataFrame in a local SQLite database in Colab.
SQL Queries:
What is the average fare by time-of-day category?
What is the distribution of tip percentages?
Which hour of the day sees the highest average trip duration?
Constraints
Use only pandas, Python standard library, and SQLite inside Google Colab.
Do not use external databases or cloud services.
Dataset
NYC Yellow Taxi Trip Data (Sample CSV, January to June 2020):
https://data.cityofnewyork.us/resource/kxp8-n2sj.csv?$limit=5000

In [1]:
import pandas as pd
import sqlite3

# Download and load dataset
url = "https://data.cityofnewyork.us/resource/kxp8-n2sj.csv?$limit=5000"
df = pd.read_csv(url)

# Preview
df.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2020-01-01T00:28:15.000,2020-01-01T00:33:03.000,1,1.2,1,N,238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1,2020-01-01T00:35:39.000,2020-01-01T00:43:04.000,1,1.2,1,N,239,238,1,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1,2020-01-01T00:47:41.000,2020-01-01T00:53:52.000,1,0.6,1,N,238,238,1,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1,2020-01-01T00:55:23.000,2020-01-01T01:00:14.000,1,0.8,1,N,238,151,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2,2020-01-01T00:01:58.000,2020-01-01T00:04:16.000,1,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [2]:
column_names_index = df.columns
print(column_names_index)

Index(['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag',
       'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')


In [3]:
# Convert datetime columns
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], errors='coerce')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], errors='coerce')

# Drop rows with missing critical fields
df = df.dropna(subset=[
    'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'trip_distance', 'fare_amount', 'total_amount'
])

# Remove invalid records
df = df[(df['trip_distance'] > 0) &
        (df['fare_amount'] > 0) &
        (df['passenger_count'] > 0)]


In [4]:
# Compute trip duration in minutes
df['trip_duration_min'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Remove extremely short/long trips
df = df[(df['trip_duration_min'] > 1) & (df['trip_duration_min'] < 180)]

# Calculate tip percentage
df['Tip_Percent'] = (df['tip_amount'] / df['total_amount']) * 100
df['Tip_Percent'] = df['Tip_Percent'].clip(lower=0)

# Categorize time of day
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['Time_of_Day'] = df['pickup_hour'].apply(get_time_of_day)


/tmp/ipython-input-1620964503.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tip_Percent'] = (df['tip_amount'] / df['total_amount']) * 100
/tmp/ipython-input-1620964503.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tip_Percent'] = df['Tip_Percent'].clip(lower=0)
/tmp/ipython-input-1620964503.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [5]:
# Create local SQLite DB
conn = sqlite3.connect("nyc_taxi_etl.db")

# Write to table
df.to_sql("nyc_taxi", conn, if_exists="replace", index=False)


4826

In [6]:
query1 = """
SELECT Time_of_Day, ROUND(AVG(fare_amount), 2) AS Avg_Fare
FROM nyc_taxi
GROUP BY Time_of_Day
ORDER BY Avg_Fare DESC;
"""

pd.read_sql(query1, conn)


,Time_of_Day,Avg_Fare
0,Night,12.19
1,Afternoon,7.13
2,Morning,4.67


In [7]:
query2 = """
SELECT
  CASE
    WHEN Tip_Percent < 5 THEN '<5%'
    WHEN Tip_Percent < 10 THEN '5-10%'
    WHEN Tip_Percent < 20 THEN '10-20%'
    ELSE '20%+'
  END AS Tip_Bin,
  COUNT(*) AS Num_Trips
FROM nyc_taxi
GROUP BY Tip_Bin
ORDER BY Num_Trips DESC;
"""

pd.read_sql(query2, conn)


,Tip_Bin,Num_Trips
0,10-20%,2275
1,<5%,1696
2,20%+,572
3,5-10%,283


In [8]:
query3 = """
SELECT pickup_hour, ROUND(AVG(trip_duration_min), 2) AS Avg_Duration
FROM nyc_taxi
GROUP BY pickup_hour
ORDER BY Avg_Duration DESC
LIMIT 5;
"""

pd.read_sql(query3, conn)


,pickup_hour,Avg_Duration
0,13,15.35
1,0,13.65
2,1,12.94
3,23,12.37
4,21,8.18


In [9]:
median_distance = df['trip_distance'].median()
print("Median trip distance:", round(median_distance, 2), "miles")

Median trip distance: 1.9 miles


In [10]:
correlation = df[['Tip_Percent', 'trip_duration_min']].corr().iloc[0, 1]
print(f"Correlation between Tip % and Duration: {correlation:.2f}")


Correlation between Tip % and Duration: -0.03


In [11]:
query4 = """
SELECT
  payment_type,
  COUNT(*) * 100.0 / (SELECT COUNT(*) FROM nyc_taxi) AS percent
FROM nyc_taxi
GROUP BY payment_type
ORDER BY percent DESC;
"""

pd.read_sql(query4, conn)


,payment_type,percent
0,1,67.985910
1,2,31.392458
2,3,0.497306
3,4,0.124327
